In [1]:
import pandas as pd

In [2]:
data = pd.read_excel('data/fs-account.xlsx', dtype = str)

In [4]:
# 금융기업 필터링
dart_code_list = pd.read_json('data/dart_code_list.json', dtype = str)
finance_name = ['스팩', '은행', '증권', '신탁', '손해', '투자']
fin_idx = dart_code_list['정식명칭'].apply(lambda x : any(nm in x for nm in finance_name))
fin_corp = dart_code_list[fin_idx]['고유번호']
fin_con = data['corp_code'].isin(fin_corp)

In [5]:
# 2542개 기업
len(data.loc[~fin_con, 'corp_code'].unique())

2542

In [6]:
# ifrs-full-assets : 2539개 기업
con1 = data['sj_div'] == 'BS'
con2 = data['account_id'] == 'ifrs-full_Assets'
data.loc[con1 & con2 & ~fin_con, :]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,currency,thstrm_add_amount,fs_div
0,20240102000351,11011,2023,01170962,BS,재무상태표,ifrs-full_Assets,자산총계,-,제 11 기,8198529250,제 10 기,7337908651,제 9 기,4823416714,7,CNY,NaN,CFS
135,20240426000544,11011,2023,01139266,BS,재무상태표,ifrs-full_Assets,자산총계,-,제 12 기,2301814360,제 11 기,1537530568,제 10 기,1520918603,7,CNY,NaN,CFS
292,20230629000113,11011,2023,00938688,BS,재무상태표,ifrs-full_Assets,자산총계,-,제 12 기,47799691000,제 11 기,43332327000,제 10 기,36179158000,17,JPY,NaN,CFS
497,20240422000050,11011,2023,01107665,BS,재무상태표,ifrs-full_Assets,자산총계,-,제 12 기,2244691084,제 11 기,2210622501,제 10 기,1912751918,7,CNY,NaN,CFS
617,20240328000298,11011,2023,00261735,BS,재무상태표,ifrs-full_Assets,자산총계,-,제 29 기,10911165157,제 28 기,11795284672,제 27 기,13462461666,7,KRW,NaN,OFS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461232,20240523000216,11011,2023,00633835,BS,재무상태표,ifrs-full_Assets,자산총계,-,제 17 기,2890585000000,제 16 기,2567697000000,제 15 기,2385236000000,7,KRW,NaN,CFS
461419,20240321001187,11011,2023,00317104,BS,재무상태표,ifrs-full_Assets,자산총계,-,제 42 기,170954344084,제 41 기,151838026925,제 40 기,140810272529,7,KRW,NaN,CFS
461563,20240321001310,11011,2023,00651901,BS,재무상태표,ifrs-full_Assets,자산총계,-,제 17 기,1358083334270,제 16 기,1227883087659,제 15 기,1009737367111,7,KRW,NaN,OFS
461689,20240320001571,11011,2023,00661847,BS,재무상태표,ifrs-full_Assets,자산총계,-,제 17 기,190295962377,제 16 기,179622909869,제 15 기,183238935517,7,KRW,NaN,CFS


In [7]:
# CashAndCashEquivalents
# BS 기준으로 2,515개 기업이 있음
con1 = data['account_id'] == 'ifrs-full_CashAndCashEquivalents'
con2 = data['sj_div'] == 'BS'
data.loc[con1 & con2 & ~fin_con, :].groupby('corp_code').count()

cash_corp_code = data.loc[con1 & con2 & ~fin_con, 'corp_code'].unique()

In [72]:
# 재고자산 : 2,311
con1 = data['sj_div'] == 'BS'
con2 = data['account_id'].str.contains('ifrs-full_Inventories') 
con3 = data['account_id'].str.contains('-표준계정코드 미사용-') & (data['account_nm'] == '재고자산')
data.loc[con1 & (con2 | con3) & ~fin_con, ['corp_code']].drop_duplicates()

inv_corp_code = data.loc[con1 & (con2|con3) & ~fin_con, 'corp_code'].unique()

In [75]:
data.loc[con1 & (con2 | con3) & ~fin_con, ['corp_code', 'account_id']].groupby('corp_code').count().sort_values('account_id', ascending=False)

,account_id
corp_code,
01806951,1
00100601,1
00100939,1
00101044,1
00101220,1
...,...
00102113,1
00101752,1
00101664,1


In [9]:
# 유동자산 : 2,506
con1 = data['sj_div'] == 'BS'
con2 = data['account_id'].str.contains('ifrs-full_CurrentAssets')
data.loc[con1 & con2 & ~fin_con, :].drop_duplicates()

ca_corp_code = data.loc[con1 & con2 & ~fin_con, 'corp_code'].unique()

In [10]:
# 비유동자산 : 2,502
con1 = data['sj_div'] == 'BS'
con2 = data['account_id'] == 'ifrs-full_NoncurrentAssets'
data.loc[con1 & con2 & ~fin_con, :].drop_duplicates()

nca_corp_code = data.loc[con1 & con2 & ~fin_con, 'corp_code'].unique()

In [12]:
# 매출채권 : 2,481
# ifrs만 있으면 ifrs
# dart_shortTerm이 같이 있으면 dart_shortTerm
con1 = data['sj_div'] == 'BS'
con2 = data['account_id'].str.contains('ifrs-full_TradeAndOtherCurrentReceivables')
con3 = data['account_id'].str.contains('ifrs-full_TradeReceivables')
con4 = data['account_id'].str.contains('ifrs-full_CurrentTradeReceivables')
con5 = data['account_id'].str.contains('dart_ShortTermTradeReceivable')
con6 = data['account_id'].str.contains('-표준계정코드 미사용-') & data['account_nm'].str.contains('매출채권')

data[con1 & (con2 | con3 | con4 | con5 | con6) & ~fin_con].groupby('corp_code').count().sort_values('reprt_code', ascending=False)

rcvb_corp_code = data.loc[con1 & (con2 | con3 | con4 | con5 | con6) & ~fin_con, 'corp_code'].unique()

In [13]:
c = set(cash_corp_code)
ca = set(ca_corp_code)
nca = set(nca_corp_code)
rcvb = set(rcvb_corp_code)
inv = set(inv_corp_code)
corp_code = c & ca & nca & rcvb & inv

In [76]:
dart_code_list[dart_code_list['고유번호'].isin(corp_code)].to_excel('data/etf_fs_target_company.xlsx', index = False)

In [77]:
dart_code_list[dart_code_list['고유번호'].isin(corp_code)].to_json('data/etf_fs_target_company.json', index = False)